<a href="https://colab.research.google.com/github/sab112/myfirstrepo/blob/master/myStockPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [224]:
#take a column as 'feature', use same column as 'label' to create a model and predict next in the sequence 
import quandl
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [225]:
quandl.ApiConfig.api_key='8sea8tfXW2CSAPzEYUoq'
df= quandl.get('XNSE/BAJAJ_AUTO')
df 

,Open,High,Low,Close,Volume,Adjustment Factor,Adjustment Type
Date,,,,,,,
2010-01-01,669.844609,669.844609,669.844609,669.844609,0.0,NaN,NaN
2010-01-04,673.738166,676.772851,658.488399,661.160448,381510.0,NaN,NaN
2010-01-05,667.554280,670.741654,652.743492,664.214219,463938.0,NaN,NaN
2010-01-06,668.012346,668.012346,650.892144,653.945915,465832.0,NaN,NaN
2010-01-07,657.705870,659.996198,639.421418,642.475188,329288.0,NaN,NaN
...,...,...,...,...,...,...,...
2018-12-25,2550.461719,2550.461719,2550.461719,2550.461719,0.0,NaN,NaN
2018-12-26,2540.211758,2574.003867,2499.305522,2559.775610,410582.0,NaN,NaN
2018-12-27,2587.202446,2587.296053,2524.204970,2562.022177,558049.0,NaN,NaN


In [226]:
cdf = df[['Close']] 
cdf 

,Close
Date,
2010-01-01,669.844609
2010-01-04,661.160448
2010-01-05,664.214219
2010-01-06,653.945915
2010-01-07,642.475188
...,...
2018-12-25,2550.461719
2018-12-26,2559.775610
2018-12-27,2562.022177


In [227]:
forecast_out = 30
cdf['prediction_col'] = cdf[['Close']].shift(-forecast_out) #new column is added with data from close column, but no first 30 in Close column(list of lists)
cdf

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Close,prediction_col
Date,,
2010-01-01,669.844609,682.804048
2010-01-04,661.160448,684.292761
2010-01-05,664.214219,686.144110
2010-01-06,653.945915,702.042804
2010-01-07,642.475188,691.869930
...,...,...
2018-12-25,2550.461719,NaN
2018-12-26,2559.775610,NaN
2018-12-27,2562.022177,NaN


In [228]:
x = np.array(cdf.drop(['prediction_col'],1))  #Close column alone, prediction_column is dropped
x = x[:-forecast_out]                         #removing last 30 rows
print(cdf)
print(x)

                  Close  prediction_col
Date                                   
2010-01-01   669.844609      682.804048
2010-01-04   661.160448      684.292761
2010-01-05   664.214219      686.144110
2010-01-06   653.945915      702.042804
2010-01-07   642.475188      691.869930
...                 ...             ...
2018-12-25  2550.461719             NaN
2018-12-26  2559.775610             NaN
2018-12-27  2562.022177             NaN
2018-12-28  2543.768822             NaN
2018-12-31  2546.249406             NaN

[2347 rows x 2 columns]
[[ 669.844609]
 [ 661.160448]
 [ 664.214219]
 ...
 [2498.229042]
 [2523.034883]
 [2508.338592]]


In [229]:
y = np.array(cdf['prediction_col'])   #prediction_column column alone, but this 1D, if you want 2D like x, give 1 more [] - cdf[['prediction_col']]
y = y[:-forecast_out]                 #removing last 30 rows
y                                     # x - feature, y - label/target. x is 2D(list of lists(but here 1D is represented in 2D)), y is 1D(list)
                                      #so that 1 close column is mapped with every prediction_col in ML algo to create a model

array([ 682.804048,  684.292761,  686.14411 , ..., 2562.022177,
       2543.768822, 2546.249406])

In [230]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)   #split the data, 80% for training, 20% for testing 

In [231]:
svr_rbf = SVR(kernel='rbf',C=1e3,gamma=0.1)        #construct/instanciate SVR algo class
svr_rbf.fit(x_train,y_train)                       #fit/create a model with feature,lable/target

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.1,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [232]:
lr = LinearRegression()                      #construct/instanciate LinearRegression algo class
lr.fit(x_train,y_train)                      #fit/create a model with feature,lable/target

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [233]:
predict_score_svr = svr_rbf.score(x_test,y_test)   #get accurancy of predition of model created with trainf data, 
                                                   #by calling score(20%test_feature, 20%test_lable)
print(predict_score_svr)
predict_score_lr = lr.score(x_test,y_test)
print(predict_score_lr)

0.8865150926756962
0.9484523687700734


In [234]:
x_forecast = np.array(cdf.drop(['prediction_col'],1))  #prediction_column column alone, but this 1D, if you want 2D like x, give 1 more [] - cdf[['prediction_col']]
x_forecast = x_forecast[-forecast_out:]                #this time take only last 30
print(x_forecast.shape)
print(x_forecast)

(30, 1)
[[2496.029279]
 [2438.133381]
 [2424.747587]
 [2424.747587]
 [2480.116097]
 [2433.453034]
 [2437.431329]
 [2549.853274]
 [2570.165982]
 [2561.086108]
 [2587.155643]
 [2587.38966 ]
 [2544.330464]
 [2608.076796]
 [2550.88295 ]
 [2564.455958]
 [2666.300318]
 [2654.786663]
 [2642.664563]
 [2659.3266  ]
 [2659.279797]
 [2679.966932]
 [2702.619814]
 [2628.904342]
 [2550.461719]
 [2550.461719]
 [2559.77561 ]
 [2562.022177]
 [2543.768822]
 [2546.249406]]


In [235]:
svr_rbf_predict = svr_rbf.predict(x_forecast)   #svr model's predict..last 30 Close column's prediction_col(label) by passing feature alone
print(svr_rbf_predict)
lr_predict = lr.predict(x_forecast)            #lr model's predict..last 30 Close column's prediction_col(label) by passing feature alone
print(lr_predict)                              #i.e. next 30 days forecast/prediction

[2547.35211842 2144.05820113 2422.38216069 2422.38216069 2514.86232106
 2591.98751032 2233.55348215 2535.53202839 2525.04185484 2530.12225376
 2452.5403076  2480.55672357 2522.31680195 2615.18947813 2508.34719385
 2556.04503778 2413.07511484 2483.47256866 2508.40887202 2632.8558525
 2630.82644966 2719.16006124 2600.26082698 2590.03598276 2517.52906105
 2517.52906105 2541.10981626 2546.45077616 2485.69337663 2560.62884054]
[2495.03412264 2439.39114717 2426.52623907 2426.52623907 2479.74017564
 2434.89292809 2438.71641436 2546.76364644 2566.28591921 2557.5593734
 2582.61445582 2582.83936644 2541.4557474  2602.72149697 2547.75325431
 2560.79809129 2658.67934749 2647.61372722 2635.96333857 2651.97700013
 2651.93201839 2671.81414795 2693.58553074 2622.73857367 2547.34841482
 2547.34841482 2556.29987125 2558.45901683 2540.91596076 2543.30001697]
